# Lecture 4: Density Matrix Renormalization Group (DMRG)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/4-DMRG/4-dmrg-ising.ipynb)

In this lecture, we implement the **Density Matrix Renormalization Group (DMRG)** algorithm to find ground states of 1D quantum systems represented as Matrix Product States (MPS).

DMRG is one of the most powerful methods for studying 1D quantum systems. It finds the ground state by:
1. Representing the wavefunction as an MPS with bond dimension $\chi$
2. Iteratively optimizing each tensor locally to minimize the energy
3. Sweeping back and forth through the chain until convergence

We will apply DMRG to the **Transverse Field Ising Model**:
$$H = J\sum_i \sigma^z_i \sigma^z_{i+1} - h\sum_i \sigma^x_i$$

---
## Setup

We need:
- `numpy`, `scipy`: numerical operations
- `einops`: cleaner tensor reshaping syntax
- `opt_einsum`: fast tensor contractions (much faster than `np.einsum`)
- `tenlib`: our small MPS/MPO library

The `tenlib` library provides many helpful functions for this exercise:

**Core classes:**
- `MPS(N, d, chi)`: Matrix Product State - creates random MPS, has normalization methods
- `MPO`: Matrix Product Operator for Hamiltonians and observables

**Pre-built operators** (from `tenlib.operators`):
- `Ising(N, h, J)`: constructs the Transverse Field Ising Hamiltonian MPO
- `Mz_global(N)`, `Mx_global(N)`: global magnetization operators

**Optimization tools** (from `tenlib.lanczos_routines`):
- `optimize_lanczos(Afunc, v_init, num_iter)`: efficient eigenvalue solver for local optimization

**Tensor utilities** (from `tenlib.utils`):
- `contract_left(A, W, B, L)`: updates left environment tensors
- `contract_right(A, W, B, R)`: updates right environment tensors

These helpers will make your DMRG implementation much cleaner!

In [ ]:
# Install dependencies if on Colab
# !pip install numpy scipy einops opt_einsum matplotlib
# !wget -q https://github.com/PhilipVinc/ComputationalQuantumPhysics/raw/main/Notebooks/4-DMRG/tenlib.tar.gz
# !tar -xzf tenlib.tar.gz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd

# Import our tensor network library
import tenlib
from tenlib import MPS, MPO
from tenlib.operators import Ising
from tenlib.lanczos_routines import optimize_lanczos
from tenlib.utils import contract_left, contract_right
from opt_einsum import contract

---
## DMRG Algorithm Overview

The DMRG algorithm works as follows:

**Initialization:**
1. Choose a bond dimension $\chi$ for the MPS
2. Initialize a random MPS and put it in right-canonical form
3. Compute all right environment tensors $R^i$ for $i \in [1, \ldots, N]$
4. Initialize the left environment $L^0$ (a trivial identity tensor)

**Left-to-Right Sweep:**
For each site $i$ from $0$ to $N-1$:
1. Form the effective Hamiltonian from $L_{i-1}$, $W_i$, and $R_{i+1}$
2. Find the optimal tensor $A_i'$ by solving the local eigenvalue problem
3. Perform SVD: $A_i' = U \Sigma V^\dagger$
4. Update: $A_i \leftarrow U$, and absorb $\Sigma V^\dagger$ into $A_{i+1}$
5. Compute the new left environment $L_i$

**Right-to-Left Sweep:**
For each site $i$ from $N-1$ to $0$:
1. Form the effective Hamiltonian from $L_{i-1}$, $W_i$, and $R_{i+1}$
2. Find optimal $A_i'$ by solving the local eigenvalue problem
3. Perform SVD: $A_i' = U \Sigma V^\dagger$
4. Update: $A_i \leftarrow V^\dagger$, and absorb $U\Sigma$ into $A_{i-1}$
5. Compute the new right environment $R_i$

**Convergence:**
- After each full sweep (left + right), compute the energy
- Repeat until the energy converges

The key diagram for the right environment update:

```
  0-+     -A--+
    |      |  |
  1-R' =  -W--R
    |      |  |
  2-+     -B--+
```

And similarly for the left environment (see `tenlib.utils.contract_left`).

---
## Part 1: Implement DMRG

Below is a skeleton for a DMRG class. Your task is to fill in the missing parts.

### Available helpers in `tenlib`:

**MPS operations:**
- `MPS(N, d, chi)` - creates a random MPS with bond dimension `chi` (already in the skeleton!)
- `MPS.normalize_right()` - puts MPS in right-canonical form
- `MPS.normalize_left()` - puts MPS in left-canonical form
- `MPS.M[i]` - access the $i$-th tensor (shape: `(chi_left, d, chi_right)`)

**Environment contractions (from `tenlib.utils`):**
- `contract_right(A, W, B, R)` - updates the right environment tensor:
  ```
    0-+     -A--+
      |      |  |
    1-R' =  -W--R
      |      |  |
    2-+     -B--+
  ```
  Where `A` and `B` are MPS tensors (use the same tensor and its conjugate), `W` is the MPO tensor, and `R` is the previous right environment.

- `contract_left(A, W, B, L)` - updates the left environment tensor (analogous structure)

**Eigenvalue solver:**
- `optimize_lanczos(Afunc, v_init, num_iter)` - finds the lowest eigenvalue and eigenvector
  - Returns: `(eigenvector, eigenvalue)`
  - `Afunc(v)` should apply the effective Hamiltonian to vector `v`

**Energy computation:**
- `MPO.expect(MPS)` - computes $\langle\psi|H|\psi\rangle$ for an MPS

### Hints for implementation:

**Environment tensors:**
- Initialize boundary tensors $R^{N}$ and $L^{0}$ as `np.ones((1,1,1))`
- Use the helper functions `contract_right` and `contract_left` to compute all environment tensors

**Local optimization:**
- The effective Hamiltonian for site $i$ is formed from $L_{i}$, $W_i$, and $R_{i+1}$
- You'll need to:
  1. Define `apply_Heff(v)` that reshapes `v` to a tensor, contracts with the environments, and returns a flattened vector
  2. Use `optimize_lanczos(apply_Heff, v_init, num_iter=10)` to find the optimal tensor
  3. Reshape the result back to tensor form

**Normalization during sweeps:**
- After finding optimal $A_i'$, do SVD: use `scipy.linalg.svd` or `numpy.linalg.svd`
- Normalize singular values: $\Sigma \to \Sigma / \|\Sigma\|$
- **Right sweep**: store $U$ as $A_i$, absorb $\Sigma V^\dagger$ into $A_{i+1}$, update $L_{i+1}$ using `contract_left`
- **Left sweep**: store $V^\dagger$ as $A_i$, absorb $U\Sigma$ into $A_{i-1}$, update $R_i$ using `contract_right`

In [ ]:
class DMRG:
    def __init__(self, H: MPO):
        """
        Initialize DMRG with a Hamiltonian MPO.
        
        Args:
            H: Matrix Product Operator for the Hamiltonian
        """
        self.MPO = H
        self.N = H.N
        self.MPS = None
        self.energy = 0.0
        
        # Environment tensors
        self.LT = [None for _ in range(self.N + 1)]  # Left tensors
        self.RT = [None for _ in range(self.N + 1)]  # Right tensors
        
    def initialize(self, chi: int):
        """
        Initialize with a random MPS of bond dimension chi.
        
        TODO:
        1. Create a random MPS with bond dimension chi
        2. Put it in right-canonical form using MPS.normalize_right()
        3. Initialize boundary tensors LT[0] and RT[N] as (1,1,1) tensors of ones
        4. Compute all right environment tensors RT[i] for i from N-1 down to 1
        """
        # TODO: Create random MPS
        self.MPS = MPS(self.N, self.MPO.d, chi)
        
        # TODO: Put in right-canonical form
        # self.MPS.normalize_right()
        
        # TODO: Initialize boundary tensors
        # self.LT[0] = ...
        # self.RT[self.N] = ...
        
        # TODO: Compute all right environment tensors
        # for i in range(self.N - 1, 0, -1):
        #     self.RT[i] = contract_right(...)
        
        pass
    
    def right_sweep(self):
        """
        Sweep from left to right, optimizing each site.
        
        TODO:
        1. For each site i from 0 to N-1:
           a. Get current tensor shape
           b. Define the effective Hamiltonian function
           c. Optimize using Lanczos
           d. Reshape result back to tensor
           e. Perform SVD and update tensors
           f. Update left environment
        """
        for i in range(self.N):
            # Get tensor shape
            shape = self.MPS.M[i].shape
            
            # TODO: Define effective Hamiltonian
            # def apply_Heff(v):
            #     M = v.reshape(shape)
            #     # Contract with L[i], W[i], R[i+1]
            #     result = contract(...)
            #     return result.ravel()
            
            # TODO: Optimize with Lanczos
            # v_init = self.MPS.M[i].ravel()
            # v_opt, energy = optimize_lanczos(apply_Heff, v_init, num_iter=10)
            # A_opt = v_opt.reshape(shape)
            
            # TODO: SVD and update
            # - Reshape A_opt to matrix form
            # - Do SVD
            # - Normalize singular values
            # - Update MPS.M[i] and MPS.M[i+1]
            # - Update LT[i+1]
            
            pass
        
        # TODO: Compute energy at the end
        # self.energy = self.MPO.expect(self.MPS)
    
    def left_sweep(self):
        """
        Sweep from right to left, optimizing each site.
        
        TODO: Similar to right_sweep but:
        - Loop from N-1 down to 0
        - After SVD, store V^dagger as A[i]
        - Absorb U*Sigma into A[i-1]
        - Update right environment RT[i]
        """
        for i in range(self.N - 1, -1, -1):
            # TODO: Implement similar to right_sweep
            pass
        
        # TODO: Compute energy
        # self.energy = self.MPO.expect(self.MPS)
    
    def dmrg_step(self):
        """Perform one full DMRG sweep (left-to-right + right-to-left)."""
        self.right_sweep()
        self.left_sweep()
        return self.energy

---
## Part 2: Test DMRG on the Ising Model

Now let's test the DMRG implementation on the transverse field Ising model.

### Tasks:

1. **Set up the Hamiltonian:**
   - Use `Ising(N, h, J)` to create the MPO
   - Choose a system size $N \approx 20$ and field $h = 0.5$

2. **Run DMRG:**
   - Initialize with bond dimension $\chi = 10$
   - Run several DMRG sweeps (5-10 should be enough)
   - Track the energy at each sweep

3. **Check convergence:**
   - Plot energy vs sweep number
   - Does it converge?

4. **Compare with exact diagonalization (optional):**
   - For small $N$ (e.g., $N=8$), compare with exact energy
   - You can compute exact energy using `scipy.sparse.linalg.eigsh` with the full Hamiltonian

In [ ]:
# System parameters
N = 20
h = 0.5
J = 1.0

# Create Hamiltonian MPO
H = Ising(N, h, J)

# Initialize DMRG
dmrg = DMRG(H)
dmrg.initialize(chi=10)

# Run DMRG sweeps
num_sweeps = 10
energies = []

for sweep in range(num_sweeps):
    energy = dmrg.dmrg_step()
    energies.append(energy)
    print(f"Sweep {sweep+1}: E = {energy:.8f}")

# Plot convergence
plt.figure(figsize=(8, 5))
plt.plot(range(1, num_sweeps+1), energies, 'o-')
plt.xlabel('Sweep number')
plt.ylabel('Energy')
plt.title(f'DMRG Convergence (N={N}, h={h}, χ={dmrg.MPS.chi})')
plt.grid(True, alpha=0.3)
plt.show()

---
## Part 3: Study the Phase Transition

The transverse field Ising model has a quantum phase transition at $h_c = 1$.

For what to do, follow the same study proposed in the second notebook.

In [ ]:
# TODO: Scan over h values and compute ground state properties
# h_values = np.linspace(0.2, 2.0, 20)
# energies = []
# mz_values = []  # Magnetization in z
# mx_values = []  # Magnetization in x
# 
# # Create magnetization operators (do this once)
# Mz_op = tenlib.operators.Mz_global(N)
# Mx_op = tenlib.operators.Mx_global(N)
# 
# for h in h_values:
#     H = Ising(N, h, J)
#     dmrg = DMRG(H)
#     dmrg.initialize(chi=10)
#     
#     # Run to convergence
#     for _ in range(10):
#         dmrg.dmrg_step()
#     
#     energies.append(dmrg.energy)
#     
#     # Compute observables
#     mz = Mz_op.expect(dmrg.MPS) / N  # Per-site magnetization
#     mx = Mx_op.expect(dmrg.MPS) / N
#     mz_values.append(mz)
#     mx_values.append(mx)
#
# # Plot results
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
# 
# ax1.plot(h_values, energies, 'o-')
# ax1.set_xlabel('h')
# ax1.set_ylabel('Ground state energy')
# ax1.axvline(1.0, color='red', linestyle='--', alpha=0.5, label='Critical point')
# ax1.legend()
# ax1.grid(True, alpha=0.3)
# 
# ax2.plot(h_values, mz_values, 'o-', label='$M_z$')
# ax2.plot(h_values, mx_values, 's-', label='$M_x$')
# ax2.set_xlabel('h')
# ax2.set_ylabel('Magnetization per site')
# ax2.axvline(1.0, color='red', linestyle='--', alpha=0.5)
# ax2.legend()
# ax2.grid(True, alpha=0.3)
# 
# plt.tight_layout()
# plt.show()